# Dog Breed Identification CNN

In [ ]:
# Watermark
%load_ext watermark
%watermark -v -m -p tensorflow,numpy

In [ ]:
# Imports
from datetime import datetime
import os
from time import time

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Setup
plt.rcParams['figure.figsize'] = (12., 8.)

In [ ]:
# Constants
processed_dir = os.path.join(os.path.pardir, 'data', 'processed')
train_dir = os.path.join(processed_dir, 'train')
test_dir = os.path.join(processed_dir, 'test')

In [ ]:
# Create the generators for the input data
input_size = (299, 299)
batch_size = 64

# - Data generators
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
test_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

# - Flow from directory generators
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=input_size, batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=input_size, batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# Build the model
# - Construct the base model
base_model = keras.applications.inception_v3.InceptionV3(
    weights='imagenet', include_top=False
)
base_model.trainable = False

# - Construct the full model
model = keras.Sequential([
    # - InceptionV3 base
    base_model,
    # - A pooling layer
    keras.layers.GlobalAveragePooling2D(),
    # - First dense layer
    keras.layers.Dense(1024, activation='relu'),
    # - Output layer
    keras.layers.Dense(120, activation='softmax')
])

model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Configure logging with TensorBoard and checkpoints
log_dir = os.path.abspath(os.path.join(
    os.path.pardir, 'logs', f"{datetime.today():%Y%m%d}-{round(time())}"
))
cp_dir = os.path.join(log_dir, 'checkpoints')
os.mkdir(log_dir)
os.mkdir(cp_dir)

cb_tboard = keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=0, update_freq='batch'
)
cb_checkpoint = keras.callbacks.ModelCheckpoint(
    os.path.join(cp_dir, 'init.{epoch:02d}-{val_loss:.2f}.hdf5'),
    save_weights_only=True, verbose=1
)
callbacks = [cb_tboard, cb_checkpoint]

In [ ]:
# Train the model
epochs = 3

history_init = model.fit_generator(
    train_generator, epochs=epochs, shuffle=True, callbacks=callbacks,
    validation_data=test_generator
)

In [ ]:
# Load from checkpoint
model.load_weights('../models/priming/init.03-0.48.hdf5')

In [ ]:
# Model tuning setup

# - Augment the data set
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    horizontal_flip=True, 
    vertical_flip=True,
    rotation_range=20
)
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=input_size, batch_size=batch_size,
    class_mode='categorical'
)

# - Unfreeze the last inception block
base_model.trainable = True
for layer in base_model.layers[:-31]:
    layer.trainable = False

# - Recompile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

# - Fit the model (again)
cb_checkpoint = keras.callbacks.ModelCheckpoint(
    os.path.join(cp_dir, 'med.{epoch:02d}-{val_loss:.2f}.hdf5'),
    save_weights_only=True, verbose=1
)
callbacks = [cb_tboard, cb_checkpoint]

In [ ]:
# Load from checkpoint
model.load_weights('../models/fine/fine.04-0.41.hdf5')

In [ ]:
# Train the model (second-round)
epochs = 10

history_med = model.fit_generator(
    train_generator, epochs=epochs, shuffle=True, callbacks=callbacks,
    validation_data=test_generator
)

In [ ]:
# Load from checkpoint
model.load_weights('../models/fine/fine.04-0.41.hdf5')

In [ ]:
# - Unfreeze last 2 inception blocks
base_model.trainable = True
for layer in base_model.layers[:-62]:
    layer.trainable = False

test_generator = train_datagen.flow_from_directory(
    test_dir, target_size=input_size, batch_size=batch_size,
    class_mode='categorical'
)

# - Recompile the model
model.compile(optimizer=keras.optimizers.SGD(lr=0.001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# - Fit the model (again)
cb_checkpoint = keras.callbacks.ModelCheckpoint(
    os.path.join(cp_dir, 'fine.{epoch:02d}-{val_loss:.2f}.hdf5'),
    save_weights_only=True, verbose=1
)
callbacks = [cb_tboard, cb_checkpoint]

In [ ]:
# Fine-tune the model
epochs = 15

history_fine = model.fit_generator(
    train_generator, epochs=epochs, shuffle=True, callbacks=callbacks,
    validation_data=test_generator
)

In [ ]:
# Combine the history objects from the 2 phases
h_acc = history_init['acc'] + history_med['acc'] + history_fine['acc']
h_vacc = history_init['val_acc'] + history_med['val_acc'] \
         + history_fine['val_acc']
h_loss = history_init['loss'] + history_med['loss'] + history_fine['loss']
h_vloss = history_init['val_loss'] + history_med['val_loss'] \
          + history_fine['val_loss']

In [ ]:
# Plot the training results
plt_x = np.arange(1, epochs+1)
plt_yt_loss = h_loss
plt_yv_loss = h_vloss
plt_yt_acc = h_acc
plt_yv_acc = h_vacc

fig, (ax_l, ax_r) = plt.subplots(1, 2, figsize=(12, 4))

ax_l.plot(plt_x, plt_yt_loss, label='Train')
ax_l.plot(plt_x, plt_yv_loss, label='Test')
ax_l.legend()
ax_l.set_title('Loss')
ax_l.set_xlabel('Epoch')
ax_l.set_ylabel('catgorical_crossentropy')

ax_r.plot(plt_x, plt_yt_acc, label='Train')
ax_r.plot(plt_x, plt_yv_acc, label='Test')
ax_r.legend()
ax_r.set_ylim(0, 1)
ax_r.set_title('Accuracy')
ax_r.set_xlabel('Epoch')
ax_r.set_ylabel('%')

fig.suptitle('Model Training', x=0.05, ha='left',
             fontsize=14, fontweight='bold');